In [ ]:
import pandas as pd
import numpy as np
import plotly.offline as pyo
import plotly.graph_objs as go


df = pd.read_csv("/home/jfractal/Documentos/dataton_2021/dataton_AGUA_2021/consumo-agua.csv")
print(df)